In [1]:
%matplotlib notebook
import uproot as ur
import matplotlib.pyplot as plt
import k3d
import numpy as np
import awkward as ak


## Get file and TTree, print branches, convert to array

In [2]:
#file = ur.open('sim_highq2.root')
#file = ur.open('klong_10GeV_45to135deg_1M.0001.root')
#file = ur.open('test_crossDivNrgCrab_35mRad_5x41_v1.0100.root')
file = ur.open('rec_mine.root')
tree = file['events']
#print(tree.keys())
ak_arrays = tree.arrays()

In [5]:
def get_vector(varname='HcalBarrelHitsReco',energy='energy'):
    E = np.array(ak.to_list(ak_arrays["%s.%s"%(varname,energy)]), dtype="O")
    x = np.array(ak.to_list(ak_arrays["%s.position.x"%varname]), dtype="O")
    y = np.array(ak.to_list(ak_arrays["%s.position.y"%varname]), dtype="O")
    z = np.array(ak.to_list(ak_arrays["%s.position.z"%varname]), dtype="O")


    #E = E/1000.0
    return E,x, y, z

In [6]:
E = {}
x = {}
y = {}
z  = {}
theta={}
phi = {}

## Get data

# get hits

In [7]:
for i in ['HcalBarrel']:#,'TrackerBarrel','VertexBarrel']:
    E[i], x[i], y[i],z[i] = get_vector("%sHitsReco"%i)
for i in ['EcalBarrel']:#,'TrackerBarrel','VertexBarrel']:
    E[i], x[i], y[i],z[i] = get_vector("%sHitsReco"%i,energy='edep')
for i in ['HcalHadronEndcap','EcalEndcap']:#,'TrackerEndcap','VertexEndcap']:
    E[i], x[i], y[i],z[i] = get_vector("%sHitsReco"%i)
for i in ['CrystalEcal']:
    E[i], x[i], y[i],z[i] = get_vector("%sHitsReco"%i)

ValueError: key "HcalBarrelHitsReco.polar.theta" does not exist (not in record)

(https://github.com/scikit-hep/awkward-1.0/blob/1.2.3/src/libawkward/util.cpp#L491)

## Get clusters

In [ ]:
for i in ['HcalBarrelClusters','HcalHadronEndcapClusters','EcalBarrelClusters','EcalEndcapClusters']:
    E[i], x[i], y[i],z[i] = get_vector("%s"%i,energy='energy')

## Plot 2D transverse view for hits in barrel

In [ ]:
#loop over events
for ievt in range(0,10):
    #HERE FILTER YOUR EVENTS IF YOU WANT
    #if(len(x['VertexBarrel'][ievt])<3): continue
    #if(len(x['TrackerBarrel'][ievt])<20): continue
    #if(len(x['HcalBarrel'][ievt])<5): continue

 
    print(ievt)
    fig = plt.figure(figsize=(7,7))

    ax = fig.add_subplot()#projection='polar')
    c = ax.scatter([0],[0], s=0.01, cmap='hsv', alpha=0.75)
    
    
    for i in ['Ecal','HcalHadron']:
        key = '%sEndcap'%i
        area = E[key][ievt]
        area = np.divide(area,sum(area))
        area = np.multiply(area,6000.0)
        if(i=='Hcal'):
            area = area*1.1
        c = ax.scatter(x[key][ievt], y[key][ievt], s=3.1,cmap='hsv', color='black',alpha=0.85,label="%s hits"%i)
    
    clusters_colors = {}
    clusters_colors['HcalHadron'] = 'blue'
    clusters_colors['Ecal'] = 'red'
    for i in ['Ecal','HcalHadron']:
        key = '%sEndcapClusters'%i
        c = ax.scatter(x[key][ievt], y[key][ievt], np.multiply(E[key][ievt],0.0851),cmap='hsv', alpha=0.55,color=clusters_colors[i],label='%s cluster'%i)

    ax.set_title( r"$\bf{ATHENA}$" +' simulation [DD4hep] \nPythia8+Geant4',fontsize=23)
    ax.set_ylabel('Y [mm]',fontsize=18)
    ax.set_xlabel('X [mm]',fontsize=18)
    plt.legend(loc='upper left')
    ax.set_ylim([-3500,3500])
    ax.set_xlim([-3500,3500])
    plt.savefig('pics/transverseview_event_%i.png'%(ievt))

## z-y view 

In [ ]:
colors = {}
colors['Tracker'] = 'k'
colors['HcalHadron'] = 'red'
colors['Ecal'] = 'blue'
colors['CrystalEcal'] ='orange'
#loop over events
for ievt in range(10):
    #HERE FILTER YOUR EVENTS IF YOU WANT
    #if(len(x['VertexBarrel'][ievt])<3): continue
    #if(len(x['TrackerEndcap'][ievt])<5): continue
    #if(len(x['HcalBarrel'][ievt])<50): continue
    

    print(ievt)
    fig = plt.figure(figsize=(8,6))

    print(ievt)
    ax = fig.add_subplot()#projection='polar')
    #c = ax.scatter([0],[0], s=1, cmap='hsv', alpha=0.75,color='black',label='Tracker')
    
    
    for i in ['Ecal','HcalHadron']:
        key = '%sEndcap'%i
        #area = E[key][ievt]
        area = E[key][ievt]
       
        area = np.divide(area,sum(area))
        area = np.multiply(area,6000.0)
        if(i=='Hcal'):
            area = area*1.1
        c = ax.scatter(z[key][ievt],y[key][ievt], s=area,cmap='hsv', alpha=0.55,color=colors[i],label=i+' hits')
    
    #area = E['HcalBarrelClusters'][ievt]
       
    #area = np.divide(area,sum(area))
    #area = np.multiply(area,600.0)
    
    #c = ax.scatter(z['HcalBarrelClusters'][ievt],y['HcalBarrelClusters'][ievt], E['HcalBarrelClusters'][ievt],cmap='hsv', 
    #               alpha=0.55,color='black',label='Hcal barrel clusters')
    c = ax.scatter(z['HcalHadronEndcapClusters'][ievt],y['HcalHadronEndcapClusters'][ievt], np.multiply(E['HcalHadronEndcapClusters'][ievt],0.05),cmap='hsv', 
                   alpha=0.55,color='green',label='Hcal hadron endcap clusters')
    c = ax.scatter(z['EcalEndcapClusters'][ievt],y['EcalEndcapClusters'][ievt], np.multiply(E['EcalEndcapClusters'][ievt],0.05),cmap='hsv', 
                   alpha=0.55,color='black',label='Ecal hadron endcap clusters')
    
    #for i in ['CrystalEcal']:
    #    key = '%s'%i
    #    area = E[key][ievt]
    #    area = np.divide(area,sum(area))
    #    area = np.multiply(area,600.0)
    #    if(i=='Hcal'):
    #        area = area*1.2
    #    c = ax.scatter(z[key][ievt], y[key][ievt], s=area,cmap='hsv', alpha=0.55,color=colors[i],label=i)

    #Plot DIRC
    ax.set_title( r"$\bf{ATHENA}$" +' simulation [DD4hep] \nPythia8+Geant4',fontsize=22)
    ax.set_ylabel('Y [mm]',fontsize=18)
    ax.set_xlabel('Z [mm]',fontsize=18)
    plt.legend()
    ax.set_ylim([-3500,3500])
    ax.set_xlim([2000,5000])
    plt.savefig('pics/longitudinalview_event_%i.png'%(ievt))

## 2D projection (theta, phi)

In [ ]:

for ievt in range(1,30):
    #ievt = 327
    #if(len(x['VertexBarrel'][ievt])<3): continue
    #fig = plt.figure(figsize=(7,7))
    #ax = fig.add_subplot(projection='polar')
    
    print(ievt)
    colors = {}
    colors['Tracker'] = 'k'
    colors['Hcal'] = 'red'
    colors['Ecal'] = 'blue'
    fig = plt.figure()
    ax = fig.add_subplot(111)

    for j in ['Barrel']:
        for i in ['Ecal','Hcal']:
            key = '%s%s'%(i,j)
            phi = 180.0/np.pi*np.arcsin(np.divide(y[key][ievt],x[key][ievt]))
            theta = 180.0/np.pi*np.arcsin(np.divide(y[key][ievt],z[key][ievt]))
            #xx = np.sin(phi)*np.cos(theta)
            #yy = np.sin(phi)*np.sin(theta)
            #zz = np.cos(phi)
             
            area = E[key][ievt]
            area = np.divide(area,sum(area))
            area = np.multiply(area,260.0)
            area=200
            if(i=='Hcal'):
                area=600
            
            ax.scatter(phi,theta,s=area,cmap='hsv',color=colors[i],alpha=0.2,label=i)
            #ax.set_ylim([-np.pi/2.0,np.pi/2.0])
            #ax.set_xlim([-np.pi/2.0,np.pi/2.0])     
            ax.set_ylabel('Polar Angle')
            ax.set_xlabel('Azimuthal Angle')
    #plt.legend()

        
    
    

## 3D viz

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.colors as colors
import matplotlib.cbook as cbook
fig = plt.figure(figsize=(10,10))

ax = plt.axes(projection='3d')
ievt =8

ax.set_title('hits')

colors = {}
colors['Vertex'] = 'black'
colors['Tracker'] = 'black'
colors['Ecal']  = 'green'
colors['HcalHadron']  = 'red'
for i in ['Ecal','HcalHadron']:
    key = '%sEndcap'%i
    area = E[key][ievt]
    area = np.divide(area,sum(area))
    area = np.multiply(area,len(E[key][ievt]))
    ax.scatter3D(z[key][ievt],x[key][ievt],y[key][ievt],s=area, color=colors[i],label=key)
plt.legend()

for i in ['Ecal','HcalHadron']:
    key = '%sEndcapClusters'%i
    area = E[key][ievt]
    #area = np.divide(area,sum(area))
    area = np.multiply(area,0.1)
    ax.scatter3D(z[key][ievt],x[key][ievt],y[key][ievt],s = area, color='black',label=key)    
#ax.scatter3D(z_ecalhit[ievt],x_ecalhit[ievt],y_ecalhit[ievt], cmap='viridis',label='Ecal')


#ax.scatter3D(z_hcalhit[ievt],x_hcalhit[ievt],y_hcalhit[ievt], cmap='copper',label='hcal')

ax.axes.set_xlim3d(left=-3500, right=3500) 
ax.axes.set_ylim3d(bottom=-3500, top=3500) 
ax.axes.set_zlim3d(bottom=-2000, top=3500) 
ax.axes.set_ylabel('x [mm]')
ax.axes.set_zlabel('y [mm]')
ax.axes.set_xlabel('z [mm]')
ax.view_init(30, 30)


